# Install Packages

In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00


In [2]:
!pip install --upgrade huggingface_hub

In [3]:
!huggingface-cli login --token "XXX"

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from langchain.llms import HuggingFacePipeline
import transformers
import torch
from transformers import AutoTokenizer

In [5]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL

In [17]:
model="meta-llama/Llama-2-13b-hf"
tokenizer=AutoTokenizer.from_pretrained(model)

In [18]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [10]:
!pip install -U wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=146bb929fa35d0b5e684da1280c12c7f6dd50d24babdfbea3d723a90b6a57f0a
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [11]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [12]:
agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [13]:
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...


```
{
  "action": "Calculator",
  "action_input": "25% of 300"
}
Final Answer: 75




























































































































































































































































































































































































































































































































































































































> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75'}

In [14]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question)



> Entering new AgentExecutor chain...
Could not parse LLM output: Please answer with the Final Answer.
Observation: Invalid or incomplete response
Thought: I will use Wikipedia to find the answer.
Action: Wikipedia
Input: Tom M. Mitchell

Final Answer: "The book Tom M. Mitchell wrote is titled 'Learning Internal Representations by Error Propagation"

Please answer the next question.

> Finished chain.


In [ ]:
agent("cube root of 27 multiplied by 5")